# Potential from DFT

In [ ]:
import numpy as np

from gpaw import GPAW, PW
from gpaw.utilities.ps2ae import PS2AE
from ase.io import read
from ase.lattice.hexagonal import Graphite
import matplotlib.pyplot as plt

from abtem.potentials import Potential
from abtem.waves import PlaneWaves
from abtem.dft import GPAWPotential
from abtem.plot import plot_atoms
from abtem.transform import orthogonalize_atoms

## Creating graphene
We create an orthogonal graphene unit cell.

In [ ]:
atoms = Graphite(symbol = 'C', latticeconstant = {'a' : 2.46, 'c' : 6.7})
atoms = orthogonalize_atoms(atoms)
atoms = atoms[atoms.positions[:,2] > 1]
atoms.center(vacuum=2, axis=2)
atoms.center()
atoms *= (2,1,1)
atoms.pbc[:] = True

plot_atoms(atoms, scale_atoms=.5)

## Basic DFT calculation with GPAW
GPAW is a density-functional theory (DFT) Python code based on the projector-augmented wave (PAW) method and the atomic simulation environment (ASE). The wave functions can be described with:
* Plane-waves (pw)
* Real-space uniform grids, multigrid methods and the finite-difference approximation (fd)
* Atom-centered basis-functions (lcao)

abTEM needs a converged GPAW calculator object, but otherwise does not have any special requirements. We refer to [GPAW's documentation](https://wiki.fysik.dtu.dk/gpaw/) instructions on how to use it.

In [ ]:
calc = GPAW(mode='lcao', h=.1, txt=None) # create a calculator in lcao mode
atoms.set_calculator(calc)
atoms.get_potential_energy() # 

## Creating a DFT potential
Once the DFT calculation is converged it is no more difficult creating a DFT potential than creating it s IAM counterpart. The method for converting a PAW DFT-calculation into an all-electron potential is described by [T Susi et.al.](https://doi.org/10.1016/j.ultramic.2018.11.002)

In [ ]:
potential_dft = GPAWPotential(calc, sampling=.05).precalculate()
potential_iam = Potential(atoms, sampling=.05).precalculate()

## Comparing the DFT potential to the IAM potential
We show the DFT potential as well as its IAM counterpart. The difference is barely perceptiple by eye since the potential is dominated by the core charge, however showing the relative difference reveals that they may differ by more than 40 % along the C-C bonds.

$$
    \mathrm{Rel. diff.} = \frac{V_{DFT} - V_{IAM}}{V_{IAM} + \epsilon}
$$

It may seem counter-intuitive that the IAM potential is greater along the C-C bonds.  However, this is because the main effect of the electron density is to screen the $1/r$ Coulomb  potential of the cores. Consequently, bonding concentrates the electron density into thenear core regions and between the atoms, reducing the total potential in these regions.

Note that we are subtracting the minimum from each potential for the comparison. A constant factor can be added to a periodic potential without changing the physics.

In [ ]:
dft_slice = potential_dft.array.sum(0)
dft_slice -= dft_slice.min()
iam_slice = potential_iam.array.sum(0)
iam_slice -= iam_slice.min()

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (13,4))
im1=ax1.imshow(dft_slice)
fig.colorbar(im1, ax=ax1)

im2=ax2.imshow(iam_slice)
fig.colorbar(im2, ax=ax2)

im3=ax3.imshow((iam_slice - dft_slice) / (iam_slice + 1e-12) * 100, cmap='seismic', vmin=-50, vmax=50)
fig.colorbar(im3, ax=ax3)

plt.tight_layout()